In [1]:
!pip install python-dotenv
!pip3 install psycopg2-binary

In [14]:
import os
from dotenv import load_dotenv
from PIL import Image
import matplotlib.pyplot as plt
import glob
import json
import pathlib

import glob
import json
import psycopg2
from psycopg2 import pool
import pandas as pd
import csv
from io import StringIO
import math
from azurecv import display_image_grid
from azurecv import (image_embedding, text_embedding, get_cosine_similarity, 
                     get_image_embedding_multiprocessing, 
                     search_by_image, search_by_text)

In [17]:
# Load environment variables
load_dotenv()
host = os.getenv("POSTGRES_HOST")
dbname = os.getenv("POSTGRES_DB_NAME")
user = os.getenv("POSTGRES_USER")
password = os.getenv("POSTGRES_PASSWORD")
sslmode = "require"
table_name = "imagevectors"

# Build a connection string from the variables
conn_string = "host={0} user={1} dbname={2} password={3} sslmode={4}".format(host, user, dbname, password, sslmode)

postgreSQL_pool = psycopg2.pool.SimpleConnectionPool(1, 20, conn_string)
if (postgreSQL_pool):
    print("Connection pool created successfully")

# Get a connection from the connection pool
conn = postgreSQL_pool.getconn()
cursor = conn.cursor()

Connection pool created successfully


In [1]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import keras
from keras import layers
from tensorflow.keras.preprocessing import image

In [3]:
# GET EMBEDDINGS
# Load image array data from .npz file
img_arr = "mnistlikedataset128x1.npz"
with np.load(img_arr) as data:
    images = data['DataX'] # col name

# Load ROI_data.csv file to get image IDs
roi_data_file = "ROI_data.csv"
roi_data = pd.read_csv(roi_data_file)
image_ids = roi_data['frameID']

# Load trained autoencoder model
autoencoder_path = 'autoencoder_model.keras'
autoencoder = keras.models.load_model(autoencoder_path)
encoder = autoencoder.layers[1]  # Get encoder part

# Get embeddings for all images
embeddings = encoder.predict(images)

# Flatten the embeddings
flattened_embeddings = embeddings.reshape(embeddings.shape[0], -1)
print(flattened_embeddings.shape)

# Normalize the embeddings
normalized_embeddings = flattened_embeddings / np.linalg.norm(flattened_embeddings, axis=1)[:, np.newaxis]

36/36 [==============================] - 6s 3ms/step
(1150, 524288)


In [13]:
# CREATE ROI_embedding.csv
import pandas as pd

# Read the CSV file
# df = pd.read_csv('ROI_data.csv')
# df['roi_id'] = df['frameID'].astype(str) + '_' + (df.index + 1).astype(str)
# df.to_csv('roi_id.csv', index=False)

data = []
for index, row in df.iterrows():
    # Extract the ROI ID from df
    roi_id = row['roi_id']
    embedding = normalized_embeddings[index]
    data.append((roi_id, embedding))

# Create a new DataFrame from the data list
embedding_df = pd.DataFrame(data, columns=['roi_id', 'embedding'])
embedding_df.to_csv('ROI_embedding.csv', index=False)
# Print the new DataFrame
print(embedding_df)

            roi_id                                          embedding
0       001_1035_1  [-0.00050622167, -0.00016938934, -0.0005967635...
1        001_115_2  [-0.00017361334, -6.870107e-05, -0.0002084288,...
2        001_115_3  [-0.0003999786, -0.00012888889, -0.00047067588...
3        001_115_4  [-0.0004040953, -0.0001317409, -0.0005031146, ...
4       001_1150_5  [-0.00019049678, -6.841586e-05, -0.00022019882...
...            ...                                                ...
1145  003_920_1146  [-0.0002441998, -9.900378e-05, -0.00031622732,...
1146  003_920_1147  [-0.0003110976, -0.000120183046, -0.0004847887...
1147  003_920_1148  [-0.00072562334, -0.00041863634, -0.0006198458...
1148  003_920_1149  [-0.0004953439, -0.00016005572, -0.00057034823...
1149  003_920_1150  [-0.0003873687, -0.0002798766, -0.0008199389, ...

[1150 rows x 2 columns]


In [16]:
embedding_df

,roi_id,embedding
0,001_1035_1,"[-0.00050622167, -0.00016938934, -0.0005967635..."
1,001_115_2,"[-0.00017361334, -6.870107e-05, -0.0002084288,..."
2,001_115_3,"[-0.0003999786, -0.00012888889, -0.00047067588..."
3,001_115_4,"[-0.0004040953, -0.0001317409, -0.0005031146, ..."
4,001_1150_5,"[-0.00019049678, -6.841586e-05, -0.00022019882..."
...,...,...
1145,003_920_1146,"[-0.0002441998, -9.900378e-05, -0.00031622732,..."
1146,003_920_1147,"[-0.0003110976, -0.000120183046, -0.0004847887..."
1147,003_920_1148,"[-0.00072562334, -0.00041863634, -0.0006198458..."
1148,003_920_1149,"[-0.0004953439, -0.00016005572, -0.00057034823..."


In [20]:
sio = StringIO()
writer = csv.writer(sio)
writer.writerows(df.values)
sio.seek(0)

cursor.execute("CREATE TEMPORARY TABLE tmp (roi_id TEXT PRIMARY KEY, embedding VECTOR(524288)) ON COMMIT DROP;")

cursor.copy_expert("COPY tmp FROM STDIN CSV", sio)

cursor.execute(f"""INSERT INTO {table_name} (roi_id, embedding)
                SELECT * FROM tmp
                ON conflict (roi_id) DO NOTHING;""")

conn.commit()

InvalidParameterValue: dimensions for type vector cannot exceed 16000
LINE 1: ...ARY TABLE tmp (roi_id TEXT PRIMARY KEY, embedding VECTOR(524...
                                                             ^
